In [1]:
val lines = sc.textFile("shakespeare.txt")

Intitializing Scala interpreter ...

Spark Web UI available at http://8762a380f160:4041
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1602068997506)
SparkSession available as 'spark'


lines: org.apache.spark.rdd.RDD[String] = shakespeare.txt MapPartitionsRDD[1] at textFile at <console>:25


In [2]:
lines.take(10).foreach(println)

1609

THE SONNETS

by William Shakespeare



                     1
  From fairest creatures we desire increase,


In [3]:
val rdd1 = lines.filter(x => x.length > 0)
rdd1.take(10).foreach(println)

1609
THE SONNETS
by William Shakespeare
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,


rdd1: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[2] at filter at <console>:26


In [4]:
val rdd2 = rdd1.map(y => y.replaceAll("""[\p{Punct}]""",""))
rdd2.take(10).foreach(println)

1609
THE SONNETS
by William Shakespeare
                     1
  From fairest creatures we desire increase
  That thereby beautys rose might never die
  But as the riper should by time decease
  His tender heir might bear his memory
  But thou contracted to thine own bright eyes
  Feedst thy lights flame with selfsubstantial fuel


rdd2: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[3] at map at <console>:26


In [5]:
val rdd3 = rdd2.flatMap(line => line.split(" "))
rdd3.take(10).foreach(println)

1609
THE
SONNETS
by
William
Shakespeare






rdd3: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[4] at flatMap at <console>:26


In [6]:
val rdd4 = rdd3.filter(_.nonEmpty)
rdd4.take(10).foreach(println)

1609
THE
SONNETS
by
William
Shakespeare
1
From
fairest
creatures


rdd4: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[5] at filter at <console>:26


In [7]:
val rdd5 = rdd4.map( w => w.toLowerCase())
rdd5.take(10).foreach(println)

1609
the
sonnets
by
william
shakespeare
1
from
fairest
creatures


rdd5: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[6] at map at <console>:26


In [8]:
val allverbs = sc.textFile("all_verbs.txt").collect()
allverbs.take(10).foreach(println)

abash
abashed
abashed
abashes
abashing
abate
abated
abated
abates
abating


allverbs: Array[String] = Array(abash, abashed, abashed, abashes, abashing, abate, abated, abated, abates, abating, abide, abode, abode, abides, abiding, absorb, absorbed, absorbed, absorbs, absorbing, accept, accepted, accepted, accepts, accepting, accompany, accompanied, accompanied, accompanies, accompanying, ache, ached, ached, aches, aching, achieve, achieved, achieved, achieves, achieving, acquire, acquired, acquired, acquires, acquiring, act, acted, acted, acts, acting, add, added, added, adds, adding, address, addressed, addressed, addresses, addressing, adjust, adjusted, adjusted, adjusts, adjusting, admire, admired, admired, admires, admiring, admit, admitted, admitted, admits, admitting, advise, advised, advised, advises, advising, afford, afforded, afforded, affords, affordi...


In [9]:
val rdd6 = rdd5.filter(z => allverbs.contains(z))
rdd6.take(10).foreach(println)

desire
increase
rose
die
bear
contracted
own
eyes
lights
making


rdd6: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[9] at filter at <console>:28


In [10]:
val rdd7 = rdd6.map(e => (e,1))
rdd7.take(10).foreach(println)

(desire,1)
(increase,1)
(rose,1)
(die,1)
(bear,1)
(contracted,1)
(own,1)
(eyes,1)
(lights,1)
(making,1)


rdd7: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[10] at map at <console>:26


In [11]:
val rdd8 = rdd7.reduceByKey((a,b) => (a+b))
rdd8.take(10).foreach(println)

(float,2)
(agree,20)
(healing,2)
(shot,45)
(guide,24)
(opening,11)
(urging,9)
(practises,1)
(surge,9)
(maintained,2)


rdd8: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[11] at reduceByKey at <console>:26


In [12]:
val rdd9 = rdd8.sortByKey()
rdd9.take(10).foreach(println)

(abate,14)
(abated,3)
(abates,1)
(abide,37)
(abides,8)
(abode,9)
(accept,30)
(accepted,3)
(accepts,2)
(accompanied,9)


rdd9: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[14] at sortByKey at <console>:26


In [13]:
val verbdict = sc.textFile("verb_dict.txt")
val rdd10 = rdd9.cartesian(verbdict)
val rdd11 = rdd10.filter(z => z._2.split(",").contains(z._1._1))
val rdd12 = rdd11.reduceByKey((a,b) => (a))
val rdd13 = rdd12.flatMap(a => a._2.split(",").slice(0,1))
val rdd14 = rdd12.map(z => z._1._2)
val rdd15 = rdd13.zip(rdd14)
val rdd16 = rdd15.reduceByKey((a,b) => (a+b))
rdd16.take(10).foreach(println)

(cleave,13)
(float,5)
(engrave,1)
(justify,11)
(offer,121)
(agree,45)
(guide,41)
(partake,9)
(improve,1)
(have,7848)


verbdict: org.apache.spark.rdd.RDD[String] = verb_dict.txt MapPartitionsRDD[16] at textFile at <console>:27
rdd10: org.apache.spark.rdd.RDD[((String, Int), String)] = CartesianRDD[17] at cartesian at <console>:28
rdd11: org.apache.spark.rdd.RDD[((String, Int), String)] = MapPartitionsRDD[18] at filter at <console>:29
rdd12: org.apache.spark.rdd.RDD[((String, Int), String)] = ShuffledRDD[19] at reduceByKey at <console>:30
rdd13: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[20] at flatMap at <console>:31
rdd14: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[21] at map at <console>:32
rdd15: org.apache.spark.rdd.RDD[(String, Int)] = ZippedPartitionsRDD2[22] at zip at <console>:33
rdd16: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[23] at reduceByKey at <console>:34


In [14]:
val rdd17 = rdd16.sortBy(_._2,false)
rdd17.take(10).foreach(println)

(be,10493)
(have,7848)
(do,6416)
(come,3610)
(make,2892)
(go,2575)
(love,2501)
(let,2384)
(say,2356)
(know,2251)


rdd17: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[28] at sortBy at <console>:26
